Keiana Dunn
Final Project: Restaurant Health Inspection Analysis using Dash 
Objective:
5/8/19


# Import library

In [1]:
import dash
import dash_table_experiments as dt
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from plotly import graph_objs as go
import json
 

# Mapbox # API key

In [2]:
mapbox_access_token = "pk.eyJ1Ijoia3RkMjAwMSIsImEiOiJjanUwZzBkZWYxYWdyNDRtcTB3M3Rqb2w2In0.HC_MMaor7bWiJEu7Ytp7pA"

# Import data

In [3]:
# Prior to bring data into jupyter lab, data was preprocessed in the terminal. This involved correcting and removing business postal codes  
df = pd.read_csv("restaurant_scores.csv").dropna() 
df.head(5)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
4,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/06/2016 12:00:00 AM,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
12,3838,CAFE PICARO,3120 16th St,San Francisco,CA,94103,37.764908,-122.422442,"(37.764908, -122.422442)",1.415543e+10,3838_20180222,02/22/2018 12:00:00 AM,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
27,64380,Michael Mina Restaurant,252 California St,San Francisco,CA,94111,37.793338,-122.399583,"(37.793338, -122.399583)",1.415535e+10,64380_20160414,04/14/2016 12:00:00 AM,86.0,Routine - Unscheduled,64380_20160414_103154,Unclean or degraded floors walls or ceilings,Low Risk
47,4787,TONY BALONEY'S,1098 HOWARD St,San Francisco,CA,94103,37.778033,-122.409430,"(37.778033, -122.40943)",1.415586e+10,4787_20160628,06/28/2016 12:00:00 AM,83.0,Routine - Unscheduled,4787_20160628_103131,Moderate risk vermin infestation,Moderate Risk
53,33700,Casa Guadalupe #3,2999 MISSION,San Francisco,CA,94110,37.749194,-122.418116,"(37.749194, -122.418116)",1.415588e+10,33700_20160412,04/12/2016 12:00:00 AM,90.0,Routine - Unscheduled,33700_20160412_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk


# Drop rows in such a way as to retain only n randomly chosen rows (in this case n is set to 1000)

In [4]:
df = df.sample(n=1000)

# Exploring the data

# Looking at index and data types

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 19709 to 35357
Data columns (total 17 columns):
business_id              1000 non-null int64
business_name            1000 non-null object
business_address         1000 non-null object
business_city            1000 non-null object
business_state           1000 non-null object
business_postal_code     1000 non-null object
business_latitude        1000 non-null float64
business_longitude       1000 non-null float64
business_location        1000 non-null object
business_phone_number    1000 non-null float64
inspection_id            1000 non-null object
inspection_date          1000 non-null object
inspection_score         1000 non-null float64
inspection_type          1000 non-null object
violation_id             1000 non-null object
violation_description    1000 non-null object
risk_category            1000 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 140.6+ KB


# Create column for individual totaling of risk category

In [6]:
def trans_risk_category(x):
    if x == 'Low Risk':
        return 1.0
    if x == 'Moderate Risk':
        return 2.0
    if x == 'High Risk':
        return 3.0
    
df['numerical_risk_category'] = df['risk_category'].apply(trans_risk_category) 

# Remove time from "Inspection Date" since all are the same time

In [7]:
import datetime as dt
df["inspection_date"] = df["inspection_date"].str.replace('12:00:00 AM','')
#df["inspection_date"].dt.strftime('%Y-%b-%d')
#df["inspection_date"].sort_values()
df["inspection_date"].max()

'12/30/2016 '

# Looking at summary stats of inspection scores

In [8]:
df["inspection_score"].describe()

count    1000.000000
mean       85.757000
std         8.009126
min        57.000000
25%        81.000000
50%        87.000000
75%        92.000000
max        98.000000
Name: inspection_score, dtype: float64

# Getting median of inspection score data

In [9]:
df["inspection_score"].median()

87.0

# Verifying inspection score datatype changed to a float

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 19709 to 35357
Data columns (total 18 columns):
business_id                1000 non-null int64
business_name              1000 non-null object
business_address           1000 non-null object
business_city              1000 non-null object
business_state             1000 non-null object
business_postal_code       1000 non-null object
business_latitude          1000 non-null float64
business_longitude         1000 non-null float64
business_location          1000 non-null object
business_phone_number      1000 non-null float64
inspection_id              1000 non-null object
inspection_date            1000 non-null object
inspection_score           1000 non-null float64
inspection_type            1000 non-null object
violation_id               1000 non-null object
violation_description      1000 non-null object
risk_category              1000 non-null object
numerical_risk_category    1000 non-null float64
dtypes: float64(5), 

# Selecting only required columns for analysis

In [11]:
map_data = df[["business_name", "business_address", "business_postal_code", "business_latitude", "business_latitude", "inspection_date", "inspection_id", "inspection_type", "violation_description", "inspection_score", "risk_category", "numerical_risk_category"]]
map_data.head(5)

,business_name,business_address,business_postal_code,business_latitude,business_latitude,inspection_date,inspection_id,inspection_type,violation_description,inspection_score,risk_category,numerical_risk_category
19709,PACIFIC CATCH,2027 CHESTNUT St,94123,37.800763,37.800763,01/17/2019,305_20190117,Routine - Unscheduled,Wiping cloths not clean or properly stored or ...,90.0,Low Risk,1.0
29332,IRMA'S PAMPANGA RESTAURANT,2901 16th St,94103,37.765123,37.765123,02/25/2019,2326_20190225,Routine - Unscheduled,Foods not protected from contamination,68.0,Moderate Risk,2.0
46757,Osha Thai Restaurant & Lounge,311 03rd St,94107,37.783662,37.783662,06/27/2017,61989_20170627,Routine - Unscheduled,Inadequately cleaned or sanitized food contact...,67.0,Moderate Risk,2.0
11816,Harvest Urban Market,191 08th St,94103,37.776384,37.776384,11/27/2018,10332_20181127,Routine - Unscheduled,Unapproved or unmaintained equipment or utensils,75.0,Low Risk,1.0
23308,VALENCIA GROCERY,1300 VALENCIA St,94110,37.752059,37.752059,10/12/2017,7646_20171012,Routine - Unscheduled,Low risk vermin infestation,98.0,Low Risk,1.0


# Total number of businesses

In [12]:
map_data.business_name.count()

1000

# Restaurants grouped by postal codes

In [13]:
zip_code_group = map_data.groupby('business_postal_code').size()
zip_code_group.count()


24

# What are all of the postal codes

In [14]:
map_data.business_postal_code.unique()

array(['94123', '94103', '94107', '94110', '94108', '94102', '94134',
       '94114', '94121', '94133', '94117', '94115', '94122', '94112',
       '94118', '94111', '94105', '94132', '94109', '94131', '94127',
       '94116', '94124', '94104'], dtype=object)

# Grouped inspection scores of restaurants

In [15]:
restaurant_scores = map_data['inspection_score'].groupby(map_data['business_name']).size()
restaurant_scores.count()

468

# Resturants grouped by Risk Category 

In [16]:
category_distribution = map_data.groupby('risk_category').size()
pd.DataFrame({'Count of Restaurants within Risk Category Totals':category_distribution.values}, 
             index=category_distribution.index)

,Count of Restaurants within Risk Category Totals
risk_category,
High Risk,140
Low Risk,479
Moderate Risk,381


# Group Restaurants by inspection scores

In [17]:
inspection_score_distribution = map_data.groupby('inspection_score').size()
pd.DataFrame({'Count of Restaurants Inspection Scores Totals':inspection_score_distribution.values}, 
             index=inspection_score_distribution.index)

,Count of Restaurants Inspection Scores Totals
inspection_score,
57.0,4
60.0,1
62.0,2
63.0,1
64.0,2
65.0,5
66.0,3
67.0,15
68.0,6


# Group restaurants by inspection type

In [18]:
inspection_type_distrubtion = map_data.groupby('inspection_type').size()
pd.DataFrame({'Count of Inspection Type Totals':inspection_type_distrubtion.values}, 
             index=inspection_type_distrubtion.index)

,Count of Inspection Type Totals
inspection_type,
Routine - Unscheduled,1000


# Violation description occuring the most

In [19]:
df['violation_description'].max()

'Wiping cloths not clean or properly stored or inadequate sanitizer'

# Violation occuring the least

In [20]:
df['violation_description'].min()

'Contaminated or adulterated food'

# List of violation descriptions

In [21]:
df.violation_description.unique()

array(['Wiping cloths not clean or properly stored or inadequate sanitizer',
       'Foods not protected from contamination',
       'Inadequately cleaned or sanitized food contact surfaces',
       'Unapproved or unmaintained equipment or utensils',
       'Low risk vermin infestation', 'Moderate risk vermin infestation',
       'Noncompliance with HAACP plan or variance',
       'Insufficient hot water or running water',
       'High risk food holding temperature',
       'Noncompliance with shell fish tags or display',
       'Unclean nonfood contact surfaces', 'Improper cooling methods',
       'Improper storage of equipment utensils or linens',
       'Permit license or inspection report not posted',
       'Inadequate food safety knowledge or lack of certified food safety manager',
       'Unclean or degraded floors walls or ceilings',
       'Inadequate dressing rooms or improper storage of personal items',
       'Inadequate and inaccessible handwashing facilities',
       'Hig

# Counts of Violation types 

In [22]:
# Violation_description groupby, count and sort them from largest violation by count to smallest
violation_description = map_data.groupby('violation_description').size()
violation_description_count = pd.DataFrame({'Count':violation_description.values},index = violation_description.index).sort_values(by = 'Count',ascending=False)
violation_description_count

,Count
violation_description,
Unclean or degraded floors walls or ceilings,110
Unapproved or unmaintained equipment or utensils,77
Inadequately cleaned or sanitized food contact surfaces,72
Moderate risk food holding temperature,68
Moderate risk vermin infestation,57
Wiping cloths not clean or properly stored or inadequate sanitizer,56
Inadequate and inaccessible handwashing facilities,55
High risk food holding temperature,51
Foods not protected from contamination,45


# Restaurants with multiple violations

In [23]:
# Looking to see if we have restaurants with multiple violations
Number_Business_Violations = map_data.groupby(['business_name','violation_description']).size()
pd.DataFrame({'Count':Number_Business_Violations.values}, index=Number_Business_Violations.index).sort_values(by='Count', ascending=False)

,,Count
business_name,violation_description,
Shanghai Dumpling King 2,Unclean or degraded floors walls or ceilings,3
SAFEWAY STORE #964,Inadequate and inaccessible handwashing facilities,3
TAQUERIA CANCUN,Unclean or degraded floors walls or ceilings,3
KIMOCHI INC,Moderate risk vermin infestation,3
21 Taste House,Unapproved or unmaintained equipment or utensils,2
KING'S BAKERY,High risk food holding temperature,2
TAQUERIA EL BUEN SABOR,Unclean nonfood contact surfaces,2
JAVA ON OCEAN,Wiping cloths not clean or properly stored or inadequate sanitizer,2
CAFE LA TAZA,Moderate risk food holding temperature,2


# Create athe text information that will be used in text box for application 

In [24]:
df['text'] = df['business_name'] +"<br>"+ df['risk_category'] +"<br>"+ df['inspection_type']+"<br>"+ df['inspection_score'].astype(str)+"<br>"+ df['violation_description'].astype(str)
df['text'].head(2)

19709    PACIFIC CATCH<br>Low Risk<br>Routine - Unsched...
29332    IRMA'S PAMPANGA RESTAURANT<br>Moderate Risk<br...
Name: text, dtype: object

#  Create color scale and color variables

In [25]:
# Color scale for inspection score map list
scl1 = [[0, 'red'],[0.5,"rgb(255, 255, 0)"], [1.0, 'green']]
# Color scale for risk category map list
scl2 = [[0, 'green'],[0.5,"rgb(255, 255, 0)"],[1.0,"rgb(255, 0, 0)"] ]
# Names for risk category color scale list
color_names= [' ','Low','Moderate','High']
# Iterate through list of color names
color_vals = list(range(len(color_names)))
# Return color Vals list
num_colors = len(color_vals)

# Plot map graph 

In [26]:
# The graph  created in Dash is more detailed with a color scale display different colored plots. I was not able to include these features 
# that I created in Dash but here is the basic code used. 
import plotly.plotly as py
import plotly.offline as offline
import matplotlib as plt 
%matplotlib inline

mapbox_access_token = "pk.eyJ1Ijoia3RkMjAwMSIsImEiOiJjanUwZzBkZWYxYWdyNDRtcTB3M3Rqb2w2In0.HC_MMaor7bWiJEu7Ytp7pA"

# Hover box information
df['text'] = df['business_name'] +"<br>"+ df['risk_category'] +"<br>"+ df['inspection_type']+"<br>"+ df['inspection_score'].astype(str)+"<br>"+ df['violation_description'].astype(str)

#Basic code to display map 1 figure
trace = go.Scattermapbox(lat = df["business_latitude"], lon = df["business_longitude"], text= df['text'], mode="markers+text", 
                         textposition="top center") 
                         
data = [trace]

#define map layout - access token (API), which location map should be centered on
layout = go.Layout(mapbox=dict(accesstoken=mapbox_access_token, center=dict(lat=37.74, lon=-122.46), zoom=11.0))

#finalize data and layout
figure = go.Figure(data = data, layout = layout)

#Display map
offline.plot(figure)

'temp-plot.html'

# Plot Histogram

In [34]:
#Basic code to display histogram 1 figure
#trace = go.Histogram(x=df['inspection_score'])

#data = [trace]

#layout = go.Layout(mapbox=dict(accesstoken=mapbox_access_token,
                #    bargap=0.01,
               #     xaxis=go.layout.XAxis(title='Inspection Scores'),
                #    yaxis=go.layout.YAxis(title='Count'))
                #    )

#figure1 = go.Figure(data = data, layout = layout)

#Display map
#offline.plot(figure1)